<a href="https://colab.research.google.com/github/YugantM/FMNIST/blob/main/main_ht.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tcn

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.utils import to_categorical
from tcn import compiled_tcn
import gzip
import sys

In [ ]:
import _pickle as cPickle

In [ ]:
import numpy as np

In [ ]:
def load_data(image_path,label_path, kind='train'):


    with gzip.open(label_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(image_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/")

In [ ]:
train_image = "gdrive/MyDrive/FashionMNIST/train-images.gz"
train_label = "gdrive/MyDrive/FashionMNIST/train-labels.gz"
test_image = "gdrive/MyDrive/FashionMNIST/test-images.gz"
test_label = "gdrive/MyDrive/FashionMNIST/test-labels.gz"

In [ ]:
(train_images, train_labels) = load_data(train_image,train_label)
(test_images, test_labels) = load_data(test_image,test_label)

In [ ]:
'''train_images = tf.reshape(train_images, shape=[-1, 28, 28])
test_images = tf.reshape(test_images, shape=[-1, 28, 28])'''

In [ ]:
plt.imshow(train_images[0].reshape([28,28]))

In [ ]:
# following command if getting SSL error while loading the dataset
#!open /Applications/Python\ 3.7/Install\ Certificates.command

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
train_images = tf.cast(train_images, tf.float32) / 255.0
test_images = tf.cast(test_images, tf.float32) / 255.0

In [ ]:
for x in range(5):
    print(class_names[train_labels[x]])
    plt.imshow(tf.reshape(train_images[x],(28,28)),cmap=plt.cm.binary)
    plt.show()

In [ ]:
train_data = train_images
test_data = test_images
model = compiled_tcn(return_sequences=False,
                         num_feat=1,
                         num_classes=10,
                         nb_filters=20,
                         kernel_size=6,
                         dilations=[2 ** i for i in range(9)],
                         nb_stacks=1,
                         max_len=train_data[0:1].shape[1],
                         use_skip_connections=True)

In [ ]:
print(model.input.shape, train_images[0:1].shape)

In [ ]:
print(f'x_train.shape = {train_images.shape}')
print(f'y_train.shape = {train_labels.shape}')
print(f'x_test.shape = {test_images.shape}')
print(f'y_test.shape = {test_labels.shape}')

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_data, train_labels, epochs=10,validation_data=(test_data, test_labels))

In [ ]:
def predict(model,image):

  accuracy = max(model.predict(tf.reshape(image,(1,784)))[0])*100
  pc = model.predict(tf.reshape(image,(1,784))).argmax(-1)[0]
  name = class_names[pc]

  print(pc,name," accuracy:", accuracy,"%")
  plt.imshow(tf.reshape(image,(28,28)),cmap=plt.cm.binary)
  plt.show()

In [ ]:
for x in test_images[:30]:
  predict(model,x)

In [ ]:
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Training History')
plt.ylabel('Loss')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()

In [ ]:
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Training Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()

In [ ]:
print("Model's training and validation accuracys are {0}% and {1}% respectively".format(history.history['accuracy'][-1]*100,history.history['val_accuracy'][-1]*100))

In [ ]:
!jupyter nbconvert --to html main_ht.ipynb